In [1]:

import stanza

import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
import pickle 
import re
import os
from tqdm import tqdm
from pathlib import Path
from os.path import expanduser
import json
import functools
import operator



In [2]:
import sys
home = expanduser("~")

# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, home+'/Documents/detect_hedges/')

from hedge_detection import hedge_percentage

2022-08-24 11:11:02 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-08-24 11:11:02 INFO: Use device: gpu
2022-08-24 11:11:02 INFO: Loading: tokenize
2022-08-24 11:11:05 INFO: Loading: pos
2022-08-24 11:11:06 INFO: Loading: lemma
2022-08-24 11:11:06 INFO: Loading: depparse
2022-08-24 11:11:06 INFO: Loading: sentiment
2022-08-24 11:11:06 INFO: Loading: constituency
2022-08-24 11:11:07 INFO: Loading: ner
2022-08-24 11:11:07 INFO: Done loading processors!


2022-08-24 11:11:07 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-08-24 11:11:07 INFO: Use device: gpu
2022-08-24 11:11:07 INFO: Loading: tokenize
2022-08-24 11:11:07 INFO: Done loading processors!


In [3]:


DATA_DIR = home+'/Documents/spinos-project/data/' 
DATA_FN = 'spinos_user_history.pkl' 
with open(os.path.join(DATA_DIR, DATA_FN), "rb") as f:
     df_hist = pickle.load(f)


DATA_DIR = home+'/Documents/spinos-project/data/' 
DATA_FN = 'spinos_annotated.pkl'
with open(os.path.join(DATA_DIR, DATA_FN), "rb") as f:
     df = pickle.load(f)
        
        
DATA_DIR = home+'/Documents/spinos-project/data/' 
DATA_FN = 'spinos_context.pkl'
with open(os.path.join(DATA_DIR, DATA_FN), "rb") as f:
     df_cont = pickle.load(f)

In [4]:
from hedge_detection_c import Load_Lexicons, Hedge_Detector
ll = Load_Lexicons()
lexicons = ll.load()

dh = Hedge_Detector(lexicons)



2022-08-24 11:11:09 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-08-24 11:11:09 INFO: Use device: gpu
2022-08-24 11:11:09 INFO: Loading: tokenize
2022-08-24 11:11:09 INFO: Loading: pos
2022-08-24 11:11:09 INFO: Loading: lemma
2022-08-24 11:11:09 INFO: Loading: depparse
2022-08-24 11:11:09 INFO: Loading: sentiment
2022-08-24 11:11:09 INFO: Loading: constituency
2022-08-24 11:11:10 INFO: Loading: ner
2022-08-24 11:11:10 INFO: Done loading processors!


2022-08-24 11:11:11 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-08-24 11:11:11 INFO: Use device: gpu
2022-08-24 11:11:11 INFO: Loading: tokenize
2022-08-24 11:11:11 INFO: Done loading processors!


In [5]:

import time
start = time.process_time()

list_hedges = []
for i, row in tqdm(df.head(100).iterrows()):
    x = row['all_text']
    try:
        list_hedges.append(hedge_percentage(x))
    except:
        print(x)
        list_hedges.append(np.nan)
        break
print(time.process_time() - start)

# df['hedge_percentage'] = list_hedges
# df['hedge_percentage'] = df['hedge_percentage'].fillna(0)


100it [00:50,  1.97it/s]

50.835649185


In [6]:

start = time.process_time()

list_hedges = []
for i, row in tqdm(df.head(100).iterrows()):
    x = row['all_text']
    try:
        list_hedges.append(dh.hedge_percentage(x))
    except:
        print(x)
        list_hedges.append(np.nan)

print(time.process_time() - start)

100it [00:50,  1.99it/s]

50.34972101299999


In [7]:
# DATA_DIR = home+'/Documents/spinos-project/data/' 
# DATA_FN = 'spinos_annotated.pkl'

# df.to_pickle(DATA_DIR+DATA_FN)